<a href="https://colab.research.google.com/github/Borgjohn715/Data-Dashboard/blob/main/seasonaldata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install nfl_data_py --upgrade

In [ ]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [ ]:
id_df = nfl.import_ids()

id_df = id_df[['pfr_id','name','gsis_id']]

snap_counts_df = nfl.import_snap_counts(list(range(2023, 2025)))

snap_counts_df = snap_counts_df[['pfr_player_id','player','position','offense_snaps','offense_pct','week']]

weekly_df = nfl.import_weekly_data(list(range(2023, 2025)))

weekly_df = weekly_df[weekly_df['position'].isin(['QB', 'RB', 'WR', 'TE'])]

merged_df = pd.merge(id_df, snap_counts_df, left_on='name', right_on='player', how='inner')
final_merged_df = pd.merge(merged_df, weekly_df, left_on='gsis_id', right_on='player_id', how='inner')
final_merged_df = final_merged_df.rename(columns={"week_x": "snap_week", "week_y": "weekly_week"})
final_matched_weeks_df = final_merged_df[final_merged_df["snap_week"] == final_merged_df["weekly_week"]]
columns_to_remove = [
    'pfr_id', 'player_display_name', 'pfr_player_id',
    'position_x', 'gsis_id', 'position_group',
    'headshot_url', 'weekly_week', 'player_name','player'
]
cleaned_df = final_matched_weeks_df.drop(columns=columns_to_remove)
cleaned_df = cleaned_df.reset_index(drop=True)
cleaned_df = cleaned_df[['player_id', 'name'] + [col for col in cleaned_df.columns if col not in ['player_id', 'name']]]
cleaned_df = cleaned_df.rename(columns={"position_y": "position", "snap_week": "week"})
cleaned_df['offense_pct'] = (cleaned_df['offense_pct'] * 100).round(2)
cleaned_df['fp/snap'] = cleaned_df['fantasy_points_ppr'] / cleaned_df['offense_snaps']
cleaned_df['game_week'] = cleaned_df['season'].astype('str') + ' - ' + cleaned_df['week'].astype('str')
cleaned_df = cleaned_df.sort_values(by=['season','week'])
cleaned_df.to_csv('snapdata241.csv', index=False)

Downcasting floats.


In [ ]:
cleaned_df.head()

,player_id,name,offense_snaps,offense_pct,week,position,recent_team,season,season_type,opponent_team,...,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,fp/snap,game_week
0,00-0039918,Caleb Williams,56.0,100.0,1,QB,CHI,2024,REG,TEN,...,0,NaN,NaN,NaN,NaN,0.0,7.22,7.22,0.128929,2024 - 1
1,00-0039910,Jayden Daniels,59.0,100.0,1,QB,WAS,2024,REG,TB,...,0,NaN,NaN,NaN,NaN,0.0,28.16,28.16,0.477288,2024 - 1
2,00-0039732,Bo Nix,69.0,100.0,1,QB,DEN,2024,REG,SEA,...,0,NaN,NaN,NaN,NaN,0.0,11.02,11.02,0.159710,2024 - 1
3,00-0039921,Trey Benson,8.0,13.0,1,RB,ARI,2024,REG,BUF,...,0,-1.25,0.032258,-0.020942,0.033727,0.0,1.80,2.80,0.350000,2024 - 1
4,00-0039373,Audric Estime,3.0,4.0,1,RB,DEN,2024,REG,SEA,...,0,NaN,NaN,NaN,NaN,0.0,1.40,1.40,0.466667,2024 - 1
